# Convolutional Neural Networks

## Project: Write an Algorithm for Landmark Classification

### A simple app

In this notebook we build a very simple app that uses our exported model.

> <img src="static_images/icons/noun-info-2558213.png" alt="?" style="width:25px"/> Note how we are not importing anything from our source code (we do not use any module from the ``src`` directory). This is because the exported model, differently from the model weights, is a standalone serialization of our model and therefore it does not need anything else. You can ship that file to anybody, and as long as they can import ``torch``, they will be able to use your model. This is very important for releasing pytorch models to production.

### Test your app
Go to a search engine for images (like Google Images) and search for images of some of the landmarks, like the Eiffel Tower, the Golden Gate Bridge, Machu Picchu and so on. Save a few examples locally, then upload them to your app to see how your model behaves!

The app will show the top 5 classes that the model think are most relevant for the picture you have uploaded

In [1]:
# Install requirements
!pip install -r requirements.txt | grep -v "already satisfied"

In [5]:
class_names = [
    "Haleakala_National_Park", "Mount_Rainier_National_Park", "Ljubljana_Castle", "Dead_Sea", "Wroclaws_Dwarves",
    "London_Olympic_Stadium", "Niagara_Falls", "Stonehenge", "Grand_Canyon", "Golden_Gate_Bridge",
    "Edinburgh_Castle", "Mount_Rushmore_National_Memorial", "Kantanagar_Temple", "Yellowstone_National_Park",
    "Terminal_Tower", "Central_Park", "Eiffel_Tower", "Changdeokgung", "Delicate_Arch", "Vienna_City_Hall",
    "Matterhorn", "Taj_Mahal", "Moscow_Raceway", "Externsteine", "Soreq_Cave", "Banff_National_Park",
    "Pont_du_Gard", "Seattle_Japanese_Garden", "Sydney_Harbour_Bridge", "Petronas_Towers", "Brooklyn_Bridge",
    "Washington_Monument", "Hanging_Temple", "Sydney_Opera_House", "Great_Barrier_Reef", "Monumento_a_la_Revolución",
    "Badlands_National_Park", "Atomium", "Forth_Bridge", "Gateway_of_India", "Stockholm_City_Hall", "Machu_Picchu",
    "Death_Valley_National_Park", "Gullfoss_Falls", "Trevi_Fountain", "Temple_of_Heaven", "Great_Wall_of_China",
    "Prague_Astronomical_Clock", "Whitby_Abbey", "Temple_of_Olympian_Zeus"
]

import json

with open('class_names.json', 'w') as f:
    json.dump(class_names, f)

In [9]:
from ipywidgets import VBox, Button, FileUpload, Output, Label
from PIL import Image
from IPython.display import display
import io
import numpy as np
import torchvision.transforms as T
import torch
import json

learn_inf = torch.jit.load("original_exported.pt")

with open('class_names.json', 'r') as f:
    class_names = json.load(f)

def on_click_classify(change):
    if not btn_upload.value:
        with out_pl:
            print("No file uploaded")
        return

    uploaded_file = next(iter(btn_upload.value.values()))
    if 'content' not in uploaded_file:
        with out_pl:
            print("Uploaded file has no content")
        return

    fn = io.BytesIO(uploaded_file['content'])

    img = Image.open(fn)
    img.load()

    out_pl.clear_output()

    with out_pl:
        ratio = img.size[0] / img.size[1]
        c = img.copy()
        c.thumbnail([ratio * 200, 200])
        display(c)

    timg = T.ToTensor()(img).unsqueeze_(0)

    with torch.no_grad():
        softmax = learn_inf(timg).data.cpu().numpy().squeeze()

    idxs = np.argsort(softmax)[::-1]

    for i in range(5):
        p = softmax[idxs[i]]
        landmark_name = class_names[idxs[i]] if idxs[i] < len(class_names) else f"Class {idxs[i]}"
        labels[i].value = f"{landmark_name} (prob: {p:.2f})"

btn_upload = FileUpload()

btn_run = Button(description="Classify")
btn_run.on_click(on_click_classify)

labels = [Label() for _ in range(5)]

out_pl = Output()
out_pl.clear_output()

wgs = [Label("Please upload a picture of a landmark"), btn_upload, btn_run, out_pl] + labels

display(VBox(wgs))


## (optional) Standalone app or web app

You can run this notebook as a standalone app on your computer by following these steps:

1. Download this notebook in a directory on your machine
2. Download the model export (for example, ``checkpoints/transfer_exported.pt``) in a subdirectory called ``checkpoints`` within the directory where you save the app.ipynb notebook
3. Install voila if you don't have it already (``pip install voila``)
4. Run your app: ``voila app.ipynb --show_tracebacks=True``
5. Customize your notebook to make your app prettier and rerun voila

You can also deploy this app as a website using Binder: https://voila.readthedocs.io/en/stable/deploy.html#deployment-on-binder

# Create your submission archive

Now that you are done with your project, please run the following cell. It will generate a file containing all the code you have written, as well as the notebooks. Please submit that file to complete your project

In [10]:
!python src/create_submit_pkg.py

executing: jupyter nbconvert --to html transfer_learning.ipynb
[NbConvertApp] Converting notebook transfer_learning.ipynb to html
[NbConvertApp] Writing 517869 bytes to transfer_learning.html
executing: jupyter nbconvert --to html cnn_from_scratch.ipynb
[NbConvertApp] Converting notebook cnn_from_scratch.ipynb to html
[NbConvertApp] Writing 1382852 bytes to cnn_from_scratch.html
executing: jupyter nbconvert --to html app.ipynb
[NbConvertApp] Converting notebook app.ipynb to html
[NbConvertApp] Writing 290248 bytes to app.html
Adding files to submission_2024-08-04T09h57m.zip
src/__init__.py
src/train.py
src/model.py
src/helpers.py
src/transfer.py
src/create_submit_pkg.py
src/data.py
src/predictor.py
src/optimization.py
transfer_learning.ipynb
cnn_from_scratch.ipynb
app.ipynb
cnn_from_scratch.html
transfer_learning.html
app.html

-------------------------------------------------------------
Done. Please submit the file submission_2024-08-04T09h57m.zip
------------------------------------